# Speciation Microservice example

This notebook performs a speciation calculation using a Docker container running the speciate function from [AqEquil](https://github.com/worm-portal/AqEquil).

## Step 0: Deploy the microservice

For cloud deployment, follow the instructions in [deployment-notes.md](https://github.com/jedick/speciation-microservice/blob/main/deployment-notes.md).

For local testing, follow the instructions in [README.md](https://github.com/jedick/speciation-microservice/blob/main/README.md).

## Step 1: Load the data

This downloads the CSV file for an aqueous speciation example from [WORM-Library](https://github.com/worm-portal/WORM-Library/tree/master/3-Aqueous-Speciation/1-Introduction-to-Aq-Speciation).
The data in this file are from [Leong et al., 2021](10.1029/2020JB020756).

If you would rather process a locally downloaded file, uncomment the bottom lines.

In [ ]:
import json

# Read the CSV file (remote file)
import requests
# URL of the remote file
url = "https://raw.githubusercontent.com/worm-portal/WORM-Library/refs/heads/master/3-Aqueous-Speciation/1-Introduction-to-Aq-Speciation/leong2021.csv"
# Fetch the file content
response = requests.get(url)
# Put the file content into a JSON object
json_data = json.dumps({"input": response.text})

# # Read the CSV file (local file)
# with open("leong2021.csv", "r") as file:
#     lines = file.readlines()
# # Convert the lines from the CSV file to JSON as a single long string in a key named 'input'
# json_data = json.dumps({"input": "".join(lines)})

## Step 2: Run speciation calculation

This sends the data to the API endpoint and stores the response.
The speciation calculation takes some time depending on the size and complexity of the input file.

NOTE: If the calculation exceeds the timeout value set in AWS Lambda, the result will be 502 (Internal Server Error).

In [ ]:
import requests

# This is the URL endpoint for the local version
url = "http://localhost:9000/2015-03-31/functions/function/invocations"

# If you have an AWS Lambda Function URL, put it here
#url = "https://<url-id>.lambda-url.<region>.on.aws/"

response = requests.post(url, json=json_data)

# Print the status code
print(f"Response status code: {response.status_code}")
# Print the error message for an unsuccessful request
if not response.status_code == 200:
    print(response.text)

## Step 3: Process the result

This converts the result to a Pandas DataFrame, using StringIO to simulate a file-like object.

In [ ]:
import pandas as pd
from io import StringIO

try:
    output = response.json() # Local
except:
    output = response.text   # AWS Lambda Function URL

df = pd.read_csv(StringIO(output[0]))
df

## Step 4: Show a plot

This shows a T-pH plot previously created by the handler function in app.py.

TODO: persist the speciation results across calls to the lambda function in order to create arbitrary plots

In [ ]:
import pickle
import base64
plot_output = pickle.loads(base64.b64decode(output[1]))
plot_output